Vector creation
===============


In [1]:
Boyle.activate("project-numy")

Resolving Hex dependencies...
Dependency resolution completed:
Unchanged:
  flow 0.15.0
  gen_stage 0.14.3
  gnuplot 1.19.96
  numy 0.1.4
All dependencies are up to date
make: Nothing to be done for 'all'.


:ok

## Mutable vectors

Numy somewhat breaks Elixir's immutable model by having mutable objects;
it is done to avoid copying big vectors and thus to improve performance.
To access a vector as immutable object use protocol called `Numy.Vc`.
To access a vector as mutable object use protocol called `Numy.Vcm`.

There are two implementation of vector:
 - `Numy.Lapack.Vector` - functions implemented as NIF C code;
   it implements both `Vc` and `Vcm` protocols.
 - `Numy.Vector` - pure Elixir, used for testing and performance
   measurements.

C `double` type is used by NIF to store vector values.

## Creating uninitialized vector

Module's function `new(n)` creates a new vector with uninitialized elements.

In [37]:
alias Numy.Vc
alias Numy.Vcm
alias Numy.Lapack, as: La

v = La.Vector.new(5) # new vector with 5 elements
Vc.data(v) # vector is not initialized, so it is having some random junk

[6.9153545207604e-310, 6.91535452096276e-310, 0.0, 0.0, 8.99726215371e-312]

An uninitialized vector can be later filled out with any values. We can fill it with 0.0.

In [38]:
Vc.assign_zeros(v)

#Vector<size=5, [0.0, 0.0, 0.0, 0.0, 0.0]>

We can assign 1.0 to all elements.

In [39]:
Vc.assign_ones(v)

#Vector<size=5, [1.0, 1.0, 1.0, 1.0, 1.0]>

Or we can assign any number.

In [40]:
Vc.assign_all(v, 42.0)

#Vector<size=5, [42.0, 42.0, 42.0, 42.0, 42.0]>

Generate random values in range `[0..1]`

In [41]:
Vc.assign_random(v)

#Vector<size=5, [0.49843955324402833, 0.5860529190504402, 0.561223499932198, 0.560083321281742, 0.6123194942029114]>

Change one element.

In [42]:
v |> Vcm.set_at!(2, 3.14)

#Vector<size=5, [0.49843955324402833, 0.5860529190504402, 3.14, 0.560083321281742, 0.6123194942029114]>

In [43]:
Vc.at(v, 2) # get value at position 2

3.14

## Create vector from a list

In [44]:
v = La.Vector.new([1,2,3,4])

#Vector<size=4, [1.0, 2.0, 3.0, 4.0]>

In [45]:
v |> Vcm.reverse! # let's reverse vector in-place

#Vector<size=4, [4.0, 3.0, 2.0, 1.0]>

In [46]:
Vc.max(v) # what is max value stored in vector?

4.0

In [47]:
Vc.min(v)

1.0

## Create vector as a copy of another vector

In [48]:
copy = La.Vector.new(v)

#Vector<size=4, [4.0, 3.0, 2.0, 1.0]>

## Create vector from a range

In [49]:
v = La.Vector.new(1..10)

#Vector<size=10, [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]>

## Create vector as concatenation of 2 other vectors

In [50]:
v1 = La.Vector.new(1..3)
v2 = La.Vector.new(4..6)
v3 = La.Vector.new(v1,v2)

#Vector<size=6, [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]>

We can also use function `concat`.

In [51]:
Vc.concat(v1,v2)

#Vector<size=6, [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]>

In [52]:
La.Vector.new([11,13,15]) |> Vc.concat(v2)

#Vector<size=6, [11.0, 13.0, 15.0, 4.0, 5.0, 6.0]>

## Save vector to a file and then load it from the file

In [53]:
v = La.Vector.new(1..100) |> Vcm.reverse!

#Vector<size=100, [100.0, 99.0, 98.0, 97.0, 96.0, 95.0, 94.0, 93.0, 92.0, 91.0, ...]>

In [54]:
La.Vector.save_to_file(v, '/home/igor/tmp/vec.bin')

:ok

In [55]:
v |> Vc.assign_random

#Vector<size=100, [0.8685350418178742, 0.5431206670213359, 0.7996093309760044, 0.09709738072095364, 0.5658408912128192, 0.701318733710286, 0.5270170455878266, 0.45451635531530055, 0.3252794386628468, 0.4187789253605494, ...]>

In [56]:
v = La.Vector.load_from_file('/home/igor/tmp/vec.bin')

#Vector<size=100, [100.0, 99.0, 98.0, 97.0, 96.0, 95.0, 94.0, 93.0, 92.0, 91.0, ...]>